In [1]:
# !pip install ipykernel
# !python -m ipykernel install --user --name tf213-p310 --display-name "Python 3.10 (TF 2.13)"
# !pip uninstall -y tensorflow tensorflow-probability keras tf-keras-nightly
# !pip install tensorflow==2.13.0 tensorflow-probability==0.19.0


In [30]:

import os
import sys
import argparse
import json
import time
import os 
import glob
import numpy as np
import six
import pickle

import tensorflow as tf
import tensorflow_probability as tfp
from tensorflow.keras.layers import BatchNormalization

# Monkey patch tf.ones_like to avoid ValueError in tfp.internal.prefer_static
def patched_ones_like(input, dtype=None, name=None):
    return tf.ones_like(input, dtype=dtype, name=name)
tf.ones_like = patched_ones_like

from imageio import imread
import scipy.io as sio

import math
from math import ceil

import pandas as pd
from sklearn.preprocessing import StandardScaler

from collections import OrderedDict, defaultdict

df = pd.read_csv("raw (FX + EQ).csv")
data = df.values

# Normalize
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# Optionally: work with returns or PCA-reduced data
# returns = np.diff(np.log(data + 1e-5), axis=0)

class FXEQDataset:
    def __init__(self, data, batch_size):
        self.original_dim = data.shape[1]  # e.g., 12
        padded_dim = 16  # make it square (4x4)
        if self.original_dim < padded_dim:
            # pad with zeros
            data = np.pad(data, ((0, 0), (0, padded_dim - self.original_dim)), mode='constant')
        elif self.original_dim > padded_dim:
            raise ValueError("Too many features to fit into 4x4 image.")

        self.data = data.reshape(-1, 4, 4, 1).astype(np.float32)  # reshape to image
        self.batch_size = batch_size
        self.idx = 0
        self.num_samples = self.data.shape[0]
        self.x_dim = [4, 4, 1]
        self.dataset_size = self.num_samples

    def next_batch(self):
        if self.idx + self.batch_size > self.num_samples:
            self.idx = 0
        batch = self.data[self.idx:self.idx + self.batch_size]
        self.idx += self.batch_size
        return batch

def lrelu(x, leak=0.2, name="lrelu"):
    return tf.maximum(x, leak * x, name=name)

def huber_loss(y_true, y_pred, delta=1.0):
    error = y_true - y_pred
    abs_error = tf.abs(error)
    quadratic = tf.minimum(abs_error, delta)
    linear = abs_error - quadratic
    return 0.5 * tf.square(quadratic) + delta * linear

tf.reset_default_graph()  # ✅ Reset before anything is built

# Set up BDCGAN

def conv_out_size(size, stride):
    co = int(math.ceil(size / float(stride)))
    return co

def kernel_sizer(size, stride):
    ko = int(math.ceil(size / float(stride)))
    if ko % 2 == 0:
        ko += 1
    return ko

def conv2d(input_, output_dim, 
           k_h=5, k_w=5, d_h=2, d_w=2, 
           stddev=0.02, name="conv2d", 
           w=None, biases=None, padding="SAME"):
    with tf.variable_scope(name):
        if w is None:
            w = tf.get_variable("w", [k_h, k_w, input_.get_shape()[-1], output_dim],
                                initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding=padding)
        if biases is None:
            biases = tf.get_variable("biases", [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.nn.bias_add(conv, biases)
        return conv

def deconv2d(input_, output_shape,
             k_h=5, k_w=5, d_h=2, d_w=2, stddev=0.02,
             name="deconv2d", w=None, biases=None, padding="SAME"):
    with tf.variable_scope(name):
        if w is None:
            w = tf.get_variable("w", [k_h, k_w, output_shape[-1], input_.get_shape()[-1]],
                                initializer=tf.truncated_normal_initializer(stddev=stddev))
        deconv = tf.nn.conv2d_transpose(input_, w,
                                        output_shape=output_shape,
                                        strides=[1, d_h, d_w, 1],
                                        padding=padding)
        if biases is None:
            biases = tf.get_variable("biases", [output_shape[-1]],
                                     initializer=tf.constant_initializer(0.0))
        deconv = tf.nn.bias_add(deconv, biases)
        return deconv



class BDCGAN(object):

    def __init__(self, x_dim, z_dim, dataset_size, batch_size=64, gf_dim=64, df_dim=64, 
                 prior_std=1.0, J=1, M=1, eta=2e-4, num_layers=4,
                 alpha=0.01, lr=0.0002, optimizer='adam', wasserstein=False, 
                 ml=False, J_d=None):

        if len(x_dim) == 1:
            c_dim = 1
            s_h = s_w = int(np.ceil(np.sqrt(x_dim[0])))  # reshape to square-like image
            self.x_dim = [s_h, s_w, 1]
        else:
            c_dim = x_dim[2]
            self.x_dim = x_dim
        self.is_grayscale = (c_dim == 1)
        self.optimizer = optimizer.lower()
        self.dataset_size = dataset_size
        self.batch_size = batch_size
        
        self.K = 2 # fake and real classes
        # self.x_dim = x_dim
        self.z_dim = z_dim

        self.gf_dim = gf_dim
        self.df_dim = df_dim
        self.c_dim = c_dim
        self.lr = lr
        
        # Bayes
        self.prior_std = prior_std
        self.num_gen = J
        self.num_disc = J_d if J_d is not None else 1
        self.num_mcmc = M
        self.eta = eta
        self.alpha = alpha
        # ML
        self.ml = ml
        if self.ml:
            assert self.num_gen == 1 and self.num_disc == 1 and self.num_mcmc == 1, "invalid settings for ML training"

        self.noise_std = np.sqrt(2 * self.alpha * self.eta)

        def get_strides(num_layers, num_pool):
            interval = int(math.floor(num_layers/float(num_pool)))
            strides = np.array([1]*num_layers)
            strides[0:interval*num_pool:interval] = 2
            return strides

        self.num_pool = 4
        self.max_num_dfs = 512
        self.gen_strides = get_strides(num_layers, self.num_pool)
        self.disc_strides = self.gen_strides
        num_dfs = np.cumprod(np.array([self.df_dim] + list(self.disc_strides)))[:-1]
        num_dfs[num_dfs >= self.max_num_dfs] = self.max_num_dfs # memory
        self.num_dfs = list(num_dfs)
        self.num_gfs = self.num_dfs[::-1]

        self.construct_from_hypers(gen_strides=self.gen_strides, disc_strides=self.disc_strides,
                                   num_gfs=self.num_gfs, num_dfs=self.num_dfs)
        
        self.build_bgan_graph()


    def construct_from_hypers(self, gen_kernel_size=5, gen_strides=[2,2,2,2],
                              disc_kernel_size=5, disc_strides=[2,2,2,2],
                              num_dfs=None, num_gfs=None):

        
        self.d_batch_norm = AttributeDict([("d_bn%i" % dbn_i, batch_norm(name='d_bn%i' % dbn_i)) for dbn_i in range(len(disc_strides))])
        self.sup_d_batch_norm = AttributeDict([("sd_bn%i" % dbn_i, batch_norm(name='sup_d_bn%i' % dbn_i)) for dbn_i in range(5)])
        self.g_batch_norm = AttributeDict([("g_bn%i" % gbn_i, batch_norm(name='g_bn%i' % gbn_i)) for gbn_i in range(len(gen_strides))])

        if num_dfs is None:
            num_dfs = [self.df_dim, self.df_dim*2, self.df_dim*4, self.df_dim*8]
            
        if num_gfs is None:
            num_gfs = [self.gf_dim*8, self.gf_dim*4, self.gf_dim*2, self.gf_dim]

        assert len(gen_strides) == len(num_gfs), "invalid hypers!"
        assert len(disc_strides) == len(num_dfs), "invalid hypers!"

        s_h, s_w = self.x_dim[0], self.x_dim[1]
        ks = gen_kernel_size
        self.gen_output_dims = OrderedDict()
        self.gen_weight_dims = OrderedDict()
        num_gfs = num_gfs + [self.c_dim]
        self.gen_kernel_sizes = [ks]
        for layer in range(len(gen_strides))[::-1]:
            self.gen_output_dims["g_h%i_out" % (layer+1)] = (s_h, s_w)
            assert gen_strides[layer] <= 2, "invalid stride"
            assert ks % 2 == 1, "invalid kernel size"
            self.gen_weight_dims["g_h%i_W" % (layer+1)] = (ks, ks, num_gfs[layer+1], num_gfs[layer])
            self.gen_weight_dims["g_h%i_b" % (layer+1)] = (num_gfs[layer+1],)
            s_h, s_w = conv_out_size(s_h, gen_strides[layer]), conv_out_size(s_w, gen_strides[layer])
            ks = kernel_sizer(ks, gen_strides[layer])
            self.gen_kernel_sizes.append(ks)


        self.gen_weight_dims.update(OrderedDict([("g_h0_lin_W", (self.z_dim, num_gfs[0] * s_h * s_w)),
                                                 ("g_h0_lin_b", (num_gfs[0] * s_h * s_w,))]))
        self.gen_output_dims["g_h0_out"] = (s_h, s_w)

        self.disc_weight_dims = OrderedDict()
        s_h, s_w = self.x_dim[0], self.x_dim[1]
        num_dfs = [self.c_dim] + num_dfs
        ks = disc_kernel_size
        self.disc_kernel_sizes = [ks]
        for layer in range(len(disc_strides)):
            assert disc_strides[layer] <= 2, "invalid stride"
            assert ks % 2 == 1, "invalid kernel size"
            self.disc_weight_dims["d_h%i_W" % layer] = (ks, ks, num_dfs[layer], num_dfs[layer+1])
            self.disc_weight_dims["d_h%i_b" % layer] = (num_dfs[layer+1],)
            s_h, s_w = conv_out_size(s_h, disc_strides[layer]), conv_out_size(s_w, disc_strides[layer])
            ks = kernel_sizer(ks, disc_strides[layer])
            self.disc_kernel_sizes.append(ks)

        self.disc_weight_dims.update(OrderedDict([("d_h_end_lin_W", (num_dfs[-1] * s_h * s_w, num_dfs[-1])),
                                                  ("d_h_end_lin_b", (num_dfs[-1],)),
                                                  ("d_h_out_lin_W", (num_dfs[-1], self.K)),
                                                  ("d_h_out_lin_b", (self.K,))]))


        for k, v in list(self.gen_output_dims.items()):
            print("%s: %s" % (k, v))
        print('****')
        for k, v in list(self.gen_weight_dims.items()):
            print("%s: %s" % (k, v))
        print('****')
        for k, v in list(self.disc_weight_dims.items()):
            print("%s: %s" % (k, v))





    def construct_nets(self):

        self.num_disc_layers = 5
        self.num_gen_layers = 5
        self.d_batch_norm = AttributeDict([("d_bn%i" % dbn_i, batch_norm(name='d_bn%i' % dbn_i)) for dbn_i in range(self.num_disc_layers)])
        self.sup_d_batch_norm = AttributeDict([("sd_bn%i" % dbn_i, batch_norm(name='sup_d_bn%i' % dbn_i)) for dbn_i in range(self.num_disc_layers)])
        self.g_batch_norm = AttributeDict([("g_bn%i" % gbn_i, batch_norm(name='g_bn%i' % gbn_i)) for gbn_i in range(self.num_gen_layers)])


        s_h, s_w = self.x_dim[0], self.x_dim[1]
        s_h2, s_w2 = conv_out_size(s_h, 2), conv_out_size(s_w, 2)
        s_h4, s_w4 = conv_out_size(s_h2, 2), conv_out_size(s_w2, 2)
        s_h8, s_w8 = conv_out_size(s_h4, 2), conv_out_size(s_w4, 2)
        s_h16, s_w16 = conv_out_size(s_h8, 2), conv_out_size(s_w8, 2)

        self.gen_output_dims = OrderedDict([("g_h0_out", (s_h16, s_w16)),
                                            ("g_h1_out", (s_h8, s_w8)),
                                            ("g_h2_out", (s_h4, s_w4)),
                                            ("g_h3_out", (s_h2, s_w2)),
                                            ("g_h4_out", (s_h, s_w))])

        
        self.gen_weight_dims = OrderedDict([("g_h0_lin_W", (self.z_dim, self.gf_dim * 8 * s_h16 * s_w16)),
                                            ("g_h0_lin_b", (self.gf_dim * 8 * s_h16 * s_w16,)),
                                            ("g_h1_W", (5, 5, self.gf_dim*4, self.gf_dim*8)),
                                            ("g_h1_b", (self.gf_dim*4,)),
                                            ("g_h2_W", (5, 5, self.gf_dim*2, self.gf_dim*4)),
                                            ("g_h2_b", (self.gf_dim*2,)),
                                            ("g_h3_W", (5, 5, self.gf_dim*1, self.gf_dim*2)),
                                            ("g_h3_b", (self.gf_dim*1,)),
                                            ("g_h4_W", (5, 5, self.c_dim, self.gf_dim*1)),
                                            ("g_h4_b", (self.c_dim,))])

        self.disc_weight_dims = OrderedDict([("d_h0_W", (5, 5, self.c_dim, self.df_dim)),
                                             ("d_h0_b", (self.df_dim,)),
                                             ("d_h1_W", (5, 5, self.df_dim, self.df_dim*2)),
                                             ("d_h1_b", (self.df_dim*2,)),
                                             ("d_h2_W", (5, 5, self.df_dim*2, self.df_dim*4)),
                                             ("d_h2_b", (self.df_dim*4,)),
                                             ("d_h3_W", (5, 5, self.df_dim*4, self.df_dim*8)),
                                             ("d_h3_b", (self.df_dim*8,)),
                                             ("d_h_end_lin_W", (self.df_dim * 8 * s_h16 * s_w16, self.df_dim*4)),
                                             ("d_h_end_lin_b", (self.df_dim*4,)),
                                             ("d_h_out_lin_W", (self.df_dim*4, self.K)),
                                             ("d_h_out_lin_b", (self.K,))])


    def _get_optimizer(self, lr):
        if self.optimizer == 'adam':
            return tf.train.AdamOptimizer(learning_rate=lr, beta1=0.5)
        elif self.optimizer == 'sgd':
            return tf.train.MomentumOptimizer(learning_rate=lr, momentum=0.5)
        else:
            raise ValueError("Optimizer must be either 'adam' or 'sgd'")    

    def initialize_wgts(self, scope_str):

        if scope_str == "generator":
            weight_dims = self.gen_weight_dims
            numz = self.num_gen
        elif scope_str == "discriminator":
            weight_dims = self.disc_weight_dims
            numz = self.num_disc
        else:
            raise RuntimeError("invalid scope!")

        param_list = []
        with tf.variable_scope(scope_str) as scope:
            for zi in range(numz):
                for m in range(self.num_mcmc):
                    wgts_ = AttributeDict()
                    for name, shape in weight_dims.items():
                        wgts_[name] = tf.get_variable("%s_%04d_%04d" % (name, zi, m),
                                                      shape, initializer=tf.random_normal_initializer(stddev=0.02))
                    param_list.append(wgts_)
            return param_list
        

    def build_bgan_graph(self):
    
        self.inputs = tf.placeholder(tf.float32,
                                     [self.batch_size] + list(self.x_dim), name='real_images')

        self.z = tf.placeholder(tf.float32, [self.batch_size, self.z_dim, self.num_gen], name='z')
        self.z_sampler = tf.placeholder(tf.float32, [self.batch_size, self.z_dim], name='z_sampler')
        
        # initialize generator weights
        self.gen_param_list = self.initialize_wgts("generator")
        self.disc_param_list = self.initialize_wgts("discriminator")
        ### build discrimitive losses and optimizers
        # prep optimizer args
        self.d_learning_rate = tf.placeholder(tf.float32, shape=[])
        
        # compile all disciminative weights
        t_vars = tf.trainable_variables()
        self.d_vars = []
        for di in range(self.num_disc):
            for m in range(self.num_mcmc):
                self.d_vars.append([var for var in t_vars if 'd_' in var.name and "_%04d_%04d" % (di, m) in var.name])

        ### build disc losses and optimizers
        self.d_losses, self.d_optims, self.d_optims_adam = [], [], []
        for di, disc_params in enumerate(self.disc_param_list):

            d_probs, d_logits, _ = self.discriminator(self.inputs, self.K, disc_params)

            constant_labels = np.zeros((self.batch_size, 2))
            constant_labels[:, 1] = 1.0  # real
            d_loss_real = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=d_logits,
                                                                                 labels=tf.constant(constant_labels)))

            d_loss_fakes = []
            for gi, gen_params in enumerate(self.gen_param_list):
                d_probs_, d_logits_, _ = self.discriminator(self.generator(self.z[:, :, gi % self.num_gen], gen_params), 
                                                            self.K, disc_params)
                constant_labels = np.zeros((self.batch_size, self.K))
                constant_labels[:, 0] = 1.0 # class label indicating it came from generator, aka fake
                d_loss_fake_ = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=d_logits_,
                                                                                      labels=tf.constant(constant_labels)))
                d_loss_fakes.append(d_loss_fake_)

            d_losses = []
            for d_loss_fake_ in d_loss_fakes:
                d_loss_ = d_loss_real * float(self.num_gen) + d_loss_fake_
                if not self.ml:
                    d_loss_ += self.disc_prior(disc_params) + self.disc_noise(disc_params)
                d_losses.append(tf.reshape(d_loss_, [1]))

            d_loss = tf.reduce_logsumexp(tf.concat(d_losses, 0))
            self.d_losses.append(d_loss)
            d_opt = self._get_optimizer(self.d_learning_rate)
            self.d_optims.append(d_opt.minimize(d_loss, var_list=self.d_vars[di]))
            d_opt_adam = tf.train.AdamOptimizer(learning_rate=self.d_learning_rate, beta1=0.5)
            self.d_optims_adam.append(d_opt_adam.minimize(d_loss, var_list=self.d_vars[di]))

        ### build generative losses and optimizers
        self.g_learning_rate = tf.placeholder(tf.float32, shape=[])
        self.g_vars = []
        for gi in range(self.num_gen):
            for m in range(self.num_mcmc):
                self.g_vars.append([var for var in t_vars if 'g_' in var.name and "_%04d_%04d" % (gi, m) in var.name])
        
        self.g_losses, self.g_optims, self.g_optims_adam = [], [], []
        for gi, gen_params in enumerate(self.gen_param_list):

            gi_losses = []
            for disc_params in self.disc_param_list:
                d_probs_, d_logits_, d_features_fake = self.discriminator(self.generator(self.z[:, :, gi % self.num_gen],
                                                                                         gen_params),
                                                                          self.K, disc_params)
                _, _, d_features_real = self.discriminator(self.inputs, self.K, disc_params)
                constant_labels = np.zeros((self.batch_size, self.K))
                constant_labels[:, 1] = 1.0 # class label indicating that this fake is real
                g_loss_ = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=d_logits_,
                                                                                 labels=tf.constant(constant_labels)))
                g_loss_ += tf.reduce_mean(huber_loss(d_features_real[-1], d_features_fake[-1]))
                if not self.ml:
                    g_loss_ += self.gen_prior(gen_params) + self.gen_noise(gen_params)
                gi_losses.append(tf.reshape(g_loss_, [1]))
                
            g_loss = tf.reduce_logsumexp(tf.concat(gi_losses, 0))
            self.g_losses.append(g_loss)
            g_opt = self._get_optimizer(self.g_learning_rate)
            self.g_optims.append(g_opt.minimize(g_loss, var_list=self.g_vars[gi]))
            g_opt_adam = tf.train.AdamOptimizer(learning_rate=self.g_learning_rate, beta1=0.5)
            self.g_optims_adam.append(g_opt_adam.minimize(g_loss, var_list=self.g_vars[gi]))

        ### build samplers
        self.gen_samplers = []
        for gi, gen_params in enumerate(self.gen_param_list):
            self.gen_samplers.append(self.generator(self.z_sampler, gen_params))


    def discriminator(self, image, K, disc_params, train=True):

        with tf.variable_scope("discriminator") as scope:

            h = image
            for layer in range(len(self.disc_strides)):
                if layer == 0:
                    h = lrelu(conv2d(h,
                                     self.disc_weight_dims["d_h%i_W" % layer][-1],
                                     name='d_h%i_conv' % layer,
                                     k_h=self.disc_kernel_sizes[layer], k_w=self.disc_kernel_sizes[layer],
                                     d_h=self.disc_strides[layer], d_w=self.disc_strides[layer],
                                     w=disc_params["d_h%i_W" % layer], biases=disc_params["d_h%i_b" % layer]))
                else:
                    h = lrelu(self.d_batch_norm["d_bn%i" % layer](conv2d(h,
                                                                         self.disc_weight_dims["d_h%i_W" % layer][-1],
                                                                         name='d_h%i_conv' % layer,
                                                                         k_h=self.disc_kernel_sizes[layer], k_w=self.disc_kernel_sizes[layer],
                                                                         d_h=self.disc_strides[layer], d_w=self.disc_strides[layer],
                                                                         w=disc_params["d_h%i_W" % layer], biases=disc_params["d_h%i_b" % layer]), train=train))

            h_end = lrelu(linear(tf.reshape(h, [self.batch_size, -1]),
                              self.df_dim*4, "d_h_end_lin",
                              matrix=disc_params.d_h_end_lin_W, bias=disc_params.d_h_end_lin_b)) # for feature norm
            h_out = linear(h_end, K,
                           'd_h_out_lin',
                           matrix=disc_params.d_h_out_lin_W, bias=disc_params.d_h_out_lin_b)
            
            return tf.nn.softmax(h_out), h_out, [h_end]
            

    def generator(self, z, gen_params):

        with tf.variable_scope("generator") as scope:

            h = linear(z, self.gen_weight_dims["g_h0_lin_W"][-1], 'g_h0_lin',
                       matrix=gen_params.g_h0_lin_W, bias=gen_params.g_h0_lin_b)
            h = tf.nn.relu(self.g_batch_norm.g_bn0(h))

            h = tf.reshape(h, [self.batch_size, self.gen_output_dims["g_h0_out"][0],
                               self.gen_output_dims["g_h0_out"][1], -1])

            for layer in range(1, len(self.gen_strides)+1):

                out_shape = [self.batch_size, self.gen_output_dims["g_h%i_out" % layer][0],
                             self.gen_output_dims["g_h%i_out" % layer][1], self.gen_weight_dims["g_h%i_W" % layer][-2]]

                h = deconv2d(h,
                             out_shape,
                             k_h=self.gen_kernel_sizes[layer-1], k_w=self.gen_kernel_sizes[layer-1],
                             d_h=self.gen_strides[layer-1], d_w=self.gen_strides[layer-1],
                             name='g_h%i' % layer,
                             w=gen_params["g_h%i_W" % layer], biases=gen_params["g_h%i_b" % layer])
                if layer < len(self.gen_strides):
                    h = tf.nn.relu(self.g_batch_norm["g_bn%i" % layer](h))

            return tf.nn.tanh(h)        


    def gen_prior(self, gen_params):
        with tf.variable_scope("generator") as scope:
            prior_loss = 0.0
            for var in list(gen_params.values()):
                nn = tf.divide(var, self.prior_std)
                prior_loss += tf.reduce_mean(tf.multiply(nn, nn))
                
        prior_loss /= self.dataset_size

        return prior_loss

    def gen_noise(self, gen_params): 
        with tf.variable_scope("generator") as scope:
            noise_loss = 0.0
            for name, var in gen_params.items():
                noise_ = tfp.distributions.Normal(loc=0., scale=self.noise_std*tf.ones(var.get_shape()))
                noise_loss += tf.reduce_sum(var * noise_.sample())
        noise_loss /= self.dataset_size
        return noise_loss

    def disc_prior(self, disc_params):
        with tf.variable_scope("discriminator") as scope:
            prior_loss = 0.0
            for var in list(disc_params.values()):
                nn = tf.divide(var, self.prior_std)
                prior_loss += tf.reduce_mean(tf.multiply(nn, nn))
                
        prior_loss /= self.dataset_size

        return prior_loss

    def disc_noise(self, disc_params): 
        with tf.variable_scope("discriminator") as scope:
            noise_loss = 0.0
            for var in list(disc_params.values()):
                noise_ = tfp.distributions.Normal(loc=0., scale=self.noise_std*tf.ones(var.get_shape()))
                noise_loss += tf.reduce_sum(var * noise_.sample())
        noise_loss /= self.dataset_size
        return noise_loss        



def one_hot_encoded(class_numbers, num_classes):
    return np.eye(num_classes, dtype=float)[class_numbers]

class AttributeDict(dict):
    def __getattr__(self, attr):
        return self[attr]
    def __setattr__(self, attr, value):
        self[attr] = value
    def __hash__(self):
        return hash(tuple(sorted(self.items())))
        
class SynthDataset():
    
    def __init__(self, x_dim=100, num_clusters=10, seed=1234):
        
        np.random.seed(seed)
        
        self.x_dim = x_dim
        self.N = 10000
        self.true_z_dim = 2
        # generate synthetic data
        self.Xs = []
        for _ in range(num_clusters):
            cluster_mean = np.random.randn(self.true_z_dim) * 5 # to make them more spread
            A = np.random.randn(self.x_dim, self.true_z_dim) * 5
            X = np.dot(np.random.randn(self.N // num_clusters, self.true_z_dim) + cluster_mean,A.T)
            self.Xs.append(X)
        X_raw = np.concatenate(self.Xs)
        self.X = (X_raw - X_raw.mean(0)) / (X_raw.std(0))
        print(self.X.shape)
        
        
    def next_batch(self, batch_size):

        rand_idx = np.random.choice(list(range(self.N)), size=(batch_size,), replace=False)
        return self.X[rand_idx]



class batch_norm(object):
    def __init__(self, name="batch_norm"):
        self.bn = BatchNormalization(momentum=0.9, epsilon=1e-5, name=name)

    def __call__(self, x, train=True):
        return self.bn(x, training=train)


def linear(input_, output_size, name=None, stddev=0.02, bias_start=0.0, matrix=None, bias=None):
    with tf.variable_scope(name or "Linear"):
        if matrix is None:
            matrix = tf.get_variable("Matrix", [input_.get_shape()[1], output_size],
                                     dtype=tf.float32,
                                     initializer=tf.random_normal_initializer(stddev=stddev))
        if bias is None:
            bias = tf.get_variable("bias", [output_size],
                                   initializer=tf.constant_initializer(bias_start))
        return tf.matmul(input_, matrix) + bias



def conv2d(input_, output_dim, 
           k_h=5, k_w=5, d_h=2, d_w=2, 
           stddev=0.02, name="conv2d", 
           w=None, biases=None, padding="SAME"):
    with tf.variable_scope(name):
        if w is None:
            w = tf.get_variable("w", [k_h, k_w, input_.get_shape()[-1], output_dim],
                                initializer=tf.truncated_normal_initializer(stddev=stddev))
        conv = tf.nn.conv2d(input_, w, strides=[1, d_h, d_w, 1], padding=padding)
        if biases is None:
            biases = tf.get_variable("biases", [output_dim], initializer=tf.constant_initializer(0.0))
        conv = tf.nn.bias_add(conv, biases)
        return conv



def b_dcgan(dataset, args):

    z_dim = args.z_dim
    x_dim = dataset.x_dim
    batch_size = args.batch_size
    dataset_size = dataset.dataset_size

    tf.reset_default_graph()
    if tf.get_default_session() is not None:
    tf.get_default_session().close()

    session = tf.InteractiveSession()


    tf.set_random_seed(args.random_seed)
    
    dcgan = BDCGAN(x_dim, z_dim, dataset_size, batch_size=batch_size,
                   J=args.J, J_d=args.J_d, M=args.M,
                   num_layers=args.num_layers,
                   lr=args.lr, optimizer=args.optimizer, gf_dim=args.gf_dim, 
                   df_dim=args.df_dim,
                   ml=(args.ml and args.J==1 and args.M==1 and args.J_d==1))
    
    print("Starting session")
    session.run(tf.global_variables_initializer())

    print("Starting training loop")
        
    num_train_iter = args.train_iter

    optimizer_dict = {"disc": dcgan.d_optims_adam,
                      "gen": dcgan.g_optims_adam}

    base_learning_rate = args.lr # for now we use same learning rate for Ds and Gs
    lr_decay_rate = args.lr_decay
    num_disc = args.J_d
    
    for train_iter in range(num_train_iter):

        if train_iter == 5000:
            print("Switching to user-specified optimizer")
            optimizer_dict = {"disc": dcgan.d_optims_adam,
                              "gen": dcgan.g_optims_adam}

        learning_rate = base_learning_rate * np.exp(-lr_decay_rate *
                                                    min(1.0, (train_iter*batch_size)/float(dataset_size)))

        image_batch = dataset.next_batch()
     

        ### compute disc losses
        batch_z = np.random.uniform(-1, 1, [batch_size, z_dim, dcgan.num_gen])
        disc_info = session.run(optimizer_dict["disc"] + dcgan.d_losses, 
                                feed_dict={dcgan.inputs: image_batch,
                                           dcgan.z: batch_z,
                                           dcgan.d_learning_rate: learning_rate})

        d_losses = disc_info[num_disc:num_disc*2]

        ### compute generative losses
        batch_z = np.random.uniform(-1, 1, [batch_size, z_dim, dcgan.num_gen])
        gen_info = session.run(optimizer_dict["gen"] + dcgan.g_losses,
                               feed_dict={dcgan.z: batch_z,
                                          dcgan.inputs: image_batch,
                                          dcgan.g_learning_rate: learning_rate})
        g_losses = [g_ for g_ in gen_info if g_ is not None]

        if train_iter > 0 and train_iter % args.n_save == 0:

            def safe_print_losses(label, losses):
                if isinstance(losses, (list, tuple)):
                    cleaned = [float(x) for x in losses if x is not None]
                    if cleaned:
                        print("{} = {}".format(label, ", ".join(["%.2f" % x for x in cleaned])))
                    else:
                        print("{}: None".format(label))
                else:
                    if losses is not None:
                        print("{} = {:.2f}".format(label, float(losses)))
                    else:
                        print("{}: None".format(label))

            safe_print_losses("Disc losses", d_losses)
            safe_print_losses("Gen losses", g_losses)

            print("saving results and samples")

            # Save results safely
            results = {
                "disc_losses": [float(x) for x in d_losses if x is not None] if d_losses else [],
                "gen_losses": [float(x) for x in g_losses if x is not None] if g_losses else []
            }

            import pandas as pd

            # Save results to CSV
            # Safely handle missing discriminator losses
            gen_losses = results.get("gen_losses", [])
            disc_losses = results.get("disc_losses", [])

            # If discriminator losses are missing, fill with NaNs
            if len(disc_losses) == 0:
                disc_losses = [np.nan] * len(gen_losses)

            # Make sure they are same length
            min_len = min(len(disc_losses), len(gen_losses))
            disc_losses = disc_losses[:min_len]
            gen_losses = gen_losses[:min_len]

            # Save as DataFrame
            df = pd.DataFrame({
                "disc_loss": disc_losses,
                "gen_loss": gen_losses
            })
            df.to_csv(os.path.join(args.out_dir, "training_losses.csv"), index=False)
            print("Saved training losses to CSV.")


            with open(os.path.join(args.out_dir, 'results_%i.json' % train_iter), 'w') as fp:
                json.dump(results, fp)

            if args.save_weights:
                var_dict = {}
                for var in tf.trainable_variables():
                    var_dict[var.name] = session.run(var) 
                np.savez_compressed(os.path.join(args.out_dir, "weights_%i.npz" % train_iter), **var_dict)
            
            print(f"Step {train_iter}: D = {np.mean(d_losses):.4f}, G = {np.mean(g_losses):.4f}")
            
            # After training loop
            saver = tf.train.Saver()
            saver.save(session, os.path.join(args.out_dir, "model.ckpt"))
            print("Training complete. Model saved to: {}".format(args.out_dir))

            print("done")



from types import SimpleNamespace

args = SimpleNamespace(
    out_dir="Results",
    n_save=100,
    z_dim=100,
    gf_dim=64,
    df_dim=96,
    data_path="raw (FX + EQ).csv",
    dataset="FXEQDataset",
    batch_size=64,
    prior_std=1.0,
    num_layers=4,
    J=1,
    J_d=1,
    M=1,
    N=128,
    train_iter=50000,
    wasserstein=False,
    ml=False,
    save_samples=False,
    save_weights=False,
    random_seed=2222,
    lr=0.005,
    lr_decay=3.0,
    optimizer="sgd"
)

    
# args = parser.parse_args()
np.random.seed(args.random_seed)
tf.set_random_seed(args.random_seed)

if not os.path.exists(args.out_dir):
    os.makedirs(args.out_dir)
args.out_dir = os.path.join(args.out_dir, f"bgan_{args.dataset}_{int(time.time())}")
os.makedirs(args.out_dir)

# Save config
import pprint
with open(os.path.join(args.out_dir, "hypers.txt"), "w") as hf:
    hf.write("Hyper settings:\n")
    hf.write("%s\n" % (pprint.pformat(args.__dict__)))


    # set seeds
np.random.seed(args.random_seed)
tf.set_random_seed(args.random_seed)

if not os.path.exists(args.out_dir):
        print("Creating %s" % args.out_dir)
        os.makedirs(args.out_dir)
args.out_dir = os.path.join(args.out_dir, "bgan_%s_%i" % (args.dataset, int(time.time())))
os.makedirs(args.out_dir)

import pprint
with open(os.path.join(args.out_dir, "hypers.txt"), "w") as hf:
        hf.write("Hyper settings:\n")
        hf.write("%s\n" % (pprint.pformat(args.__dict__)))

if args.dataset.lower() == "fxeqdataset":
        df = pd.read_csv(args.data_path)
        dataset = FXEQDataset(data=df.values, batch_size=args.batch_size)
else:
        raise RuntimeError("invalid dataset %s" % args.dataset)

        


df = pd.read_csv(args.data_path)
dataset = FXEQDataset(data=df.values, batch_size=args.batch_size)

b_dcgan(dataset, args)



class Generator2(tf.keras.Model):
    def __init__(self, z_dim, output_dim, hidden_units=[128, 256, 512]):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        self.hidden_layers = [tf.keras.layers.Dense(units, activation='relu') for units in hidden_units]
        self.out = tf.keras.layers.Dense(output_dim, activation='tanh')

    def call(self, z, training=False):
        x = z
        for layer in self.hidden_layers:
            x = layer(x, training=training)
        return self.out(x)


class Discriminator2(tf.keras.Model):
    def __init__(self, input_dim, hidden_units=[512, 256]):
        super(Discriminator, self).__init__()
        self.hidden_layers = [tf.keras.layers.Dense(units, activation='leaky_relu') for units in hidden_units]
        self.out = tf.keras.layers.Dense(1)  # logits for binary classification (real vs fake)

    def call(self, x, training=False):
        for layer in self.hidden_layers:
            x = layer(x, training=training)
        return self.out(x)

def add_gaussian_noise(x, stddev):
    noise = tfp.distributions.Normal(loc=0., scale=stddev).sample(sample_shape=tf.shape(x))
    return x + noise

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_logits, fake_logits):
    real_loss = cross_entropy(tf.ones_like(real_logits), real_logits)
    fake_loss = cross_entropy(tf.zeros_like(fake_logits), fake_logits)
    return real_loss + fake_loss

def generator_loss(fake_logits):
    return cross_entropy(tf.ones_like(fake_logits), fake_logits)

generator = Generator(z_dim=100, output_dim=scaled_data.shape[1])
discriminator = Discriminator(input_dim=scaled_data.shape[1])

g_optimizer = tf.keras.optimizers.Adam(1e-4)
d_optimizer = tf.keras.optimizers.Adam(1e-4)

@tf.function
def train_step(real_data):
    noise = tf.random.normal([real_data.shape[0], 100])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        fake_data = generator(noise, training=True)

        real_output = discriminator(real_data, training=True)
        fake_output = discriminator(fake_data, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    g_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    d_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss



C:\Users\dullz\anaconda3\envs\bgan_env36\lib\site-packages\tensorflow_core\python\client\session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


g_h4_out: (4, 4)
g_h3_out: (2, 2)
g_h2_out: (1, 1)
g_h1_out: (1, 1)
g_h0_out: (1, 1)
****
g_h4_W: (5, 5, 1, 96)
g_h4_b: (1,)
g_h3_W: (3, 3, 96, 192)
g_h3_b: (96,)
g_h2_W: (3, 3, 192, 384)
g_h2_b: (192,)
g_h1_W: (3, 3, 384, 512)
g_h1_b: (384,)
g_h0_lin_W: (100, 512)
g_h0_lin_b: (512,)
****
d_h0_W: (5, 5, 1, 96)
d_h0_b: (96,)
d_h1_W: (3, 3, 96, 192)
d_h1_b: (192,)
d_h2_W: (3, 3, 192, 384)
d_h2_b: (384,)
d_h3_W: (3, 3, 384, 512)
d_h3_b: (512,)
d_h_end_lin_W: (512, 512)
d_h_end_lin_b: (512,)
d_h_out_lin_W: (512, 2)
d_h_out_lin_b: (2,)
Starting session
Starting training loop
Disc losses = 0.03
Gen losses = 4.39
saving results and samples
Saved training losses to CSV.
Step 100: D = 0.0329, G = 4.3927
Training complete. Model saved to: Results\bgan_FXEQDataset_1748028009\bgan_FXEQDataset_1748028009
done
Disc losses = 0.01
Gen losses = 6.42
saving results and samples
Saved training losses to CSV.
Step 200: D = 0.0086, G = 6.4183
Training complete. Model saved to: Results\bgan_FXEQDataset_17480

NameError: name 'Generator' is not defined

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# Update path to your actual result folder
result_dir = "Results/bgan_FXEQDataset_1748028009/bgan_FXEQDataset_1748028009"
loss_path = os.path.join(result_dir, "training_losses.csv")

loss_df = pd.read_csv(loss_path)
plt.figure(figsize=(10, 5))
plt.plot(loss_df['disc_loss'], label='Discriminator Loss')
plt.plot(loss_df['gen_loss'], label='Generator Loss')
plt.title("Training Loss Curves")
plt.xlabel("Save Step (every 100 iterations)")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import numpy as np
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from sklearn.preprocessing import StandardScaler

# Load trained model
ckpt_path = os.path.join(result_dir, "model.ckpt")

# Load and normalize original data (for inverse scaling later)
df_real = pd.read_csv("raw (FX + EQ).csv").drop(columns=["Date"], errors="ignore").dropna()
real_values = df_real.values
scaler = StandardScaler()
scaler.fit(real_values)

# Generate samples
tf.reset_default_graph()
sess = tf.InteractiveSession()

# Rebuild model exactly as trained
dataset_size = real_values.shape[0]
x_dim = [4, 4, 1]  # from g_h4_out size
z_dim = 100
batch_size = 64

model = BDCGAN(x_dim=x_dim, z_dim=z_dim, dataset_size=dataset_size, batch_size=batch_size)
saver = tf.train.Saver()
saver.restore(sess, ckpt_path)

# Generate synthetic samples
z_input = np.random.uniform(-1, 1, [batch_size, z_dim])
synthetic_imgs = sess.run(model.gen_samplers[0], feed_dict={model.z_sampler: z_input})

# Flatten and inverse scale
synthetic_flat = synthetic_imgs.reshape(batch_size, -1)
synthetic_scaled_back = scaler.inverse_transform(synthetic_flat)


In [ ]:
import seaborn as sns

# Select a few dimensions (features) to compare
features_to_plot = [0, 1, 2]  # Adjust to match your columns

plt.figure(figsize=(15, 5))
for i, col in enumerate(features_to_plot):
    plt.subplot(1, len(features_to_plot), i+1)
    sns.kdeplot(real_values[:, col], label='Real', fill=True)
    sns.kdeplot(synthetic_scaled_back[:, col], label='Synthetic', fill=True)
    plt.title(f'Distribution - Feature {col}')
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import os
import json
import time
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tensorflow.compat.v1.keras.layers import Dense
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()-

# ----------------------- Data Preprocessing -----------------------

df = pd.read_csv("raw (FX + EQ).csv")
df = df.drop(columns=["Date"], errors="ignore")  # Drop non-numeric columns
df = df.dropna(axis=0)  # Drop rows with NaNs
data = df.values

scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)

# ----------------------- Dataset Class -----------------------

class FXEQDataset:
    def __init__(self, data, batch_size):
        self.data = data
        self.batch_size = batch_size
        self.idx = 0
        self.num_samples = data.shape[0]

        # ✅ Add these two lines:
        self.x_dim = [data.shape[1]]  # or data.shape[1:] if using multi-dimensional input
        self.dataset_size = self.num_samples

    def next_batch(self):
        if self.idx + self.batch_size > self.num_samples:
            self.idx = 0
        batch = self.data[self.idx:self.idx + self.batch_size]
        self.idx += self.batch_size
        return batch


# ----------------------- Minimal BayesGAN Mockup -----------------------

class MiniGAN:
    def __init__(self, x_dim, z_dim, batch_size, lr):
        self.x_dim = x_dim
        self.z_dim = z_dim
        self.batch_size = batch_size
        self.lr = lr
        self._build_model()

    def _build_generator(self, z):
        with tf.variable_scope("generator", reuse=tf.AUTO_REUSE):
            h1 = Dense(128, activation=tf.nn.relu)(z)
            out = tf.layers.dense(h1, self.x_dim)
        return out

    def _build_discriminator(self, x):
        with tf.variable_scope("discriminator", reuse=tf.AUTO_REUSE):
            h1 = Dense(128, activation=tf.nn.relu)(z)
            logits = tf.layers.dense(h1, 1)
        return logits

    def _build_model(self):
        self.z = tf.placeholder(tf.float32, [None, self.z_dim], name="z")
        self.x = tf.placeholder(tf.float32, [None, self.x_dim], name="x")

        self.fake_x = self._build_generator(self.z)
        real_logits = self._build_discriminator(self.x)
        fake_logits = self._build_discriminator(self.fake_x)

        # Losses
        self.d_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=real_logits, labels=tf.ones_like(real_logits))) + \
                      tf.reduce_mean(
                          tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_logits, labels=tf.zeros_like(fake_logits)))

        self.g_loss = tf.reduce_mean(
            tf.nn.sigmoid_cross_entropy_with_logits(logits=fake_logits, labels=tf.ones_like(fake_logits)))

        # Variables
        all_vars = tf.trainable_variables()
        self.d_vars = [v for v in all_vars if "discriminator" in v.name]
        self.g_vars = [v for v in all_vars if "generator" in v.name]

        # Optimizers
        self.d_opt = tf.train.AdamOptimizer(self.lr).minimize(self.d_loss, var_list=self.d_vars)
        self.g_opt = tf.train.AdamOptimizer(self.lr).minimize(self.g_loss, var_list=self.g_vars)

# ----------------------- Training Function -----------------------

def train_gan(data, args):
    tf.reset_default_graph()
    dataset = FXEQDataset(data, batch_size=args["batch_size"])
    if tf.get_default_session() is not None:
    tf.get_default_session().close()

    session = tf.InteractiveSession()

    tf.set_random_seed(args["random_seed"])

    model = MiniGAN(x_dim=data.shape[1],
                    z_dim=args["z_dim"],
                    batch_size=args["batch_size"],
                    lr=args["lr"])

    session.run(tf.global_variables_initializer())
    print("Training started...")

    for step in range(args["train_iter"]):
        batch_x, _ = dataset.next_batch()
        batch_z = np.random.uniform(-1, 1, [args["batch_size"], args["z_dim"]])

        # Train discriminator
        _ = session.run(model.d_opt, feed_dict={model.x: batch_x, model.z: batch_z})

        # Train generator
        _ = session.run(model.g_opt, feed_dict={model.z: batch_z})

        if step % args["n_save"] == 0:
            d_loss_val, g_loss_val = session.run([model.d_loss, model.g_loss],
                                                 feed_dict={model.x: batch_x, model.z: batch_z})
            print(f"Step {step}: D_loss = {d_loss_val:.4f}, G_loss = {g_loss_val:.4f}")

    print("Training complete.")
    session.close()

# ----------------------- Run -----------------------

args = {
    "batch_size": 64,
    "z_dim": 100,
    "lr": 0.001,
    "train_iter": 1000,
    "n_save": 100,
    "random_seed": 42,
}

train_gan(scaled_data, args)



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 176
Traceback (most recent call last):
  File "C:\Users\dullz\.conda\envs\tf1gan\lib\site-packages\IPython\core\interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\dullz\AppData\Local\Temp\ipykernel_24836\3617853927.py", line 7, in <module>
    from tensorflow.compat.v1.keras.layers import Dense
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 959, in _find_and_load_unlocked
  File "C:\Users\dullz\.conda\envs\tf1gan\lib\site-packages\tensorflow\__init__.py", line 50, in __getattr__
    module = self._load()
  File "C:\Users\dullz\.conda\envs\tf1gan\lib\site-packages\tensorflow\__init__.py", line 44, in _load
    module = _importlib.import_module(self.__name__)
  File "C:\Users\dullz\.conda\envs\tf1gan\lib\importlib\__init__.py", line 127, in import_module
    return _bo